In [203]:
#pip install creme
import matplotlib.pyplot as plt

In [204]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Sk-Learn

In [ ]:
pip install -U scikit-multiflow

In [206]:
from skmultiflow.data import SEAGenerator
from skmultiflow.trees import HoeffdingTreeClassifier

In [207]:
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import pandas as pd
from skmultiflow.data import DataStream

In [246]:
data1 = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/data_1.csv')
data2=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_2.csv')

In [248]:
data1 = data1.drop_duplicates()
data2=data2.drop_duplicates()

In [250]:
x1=data1.drop(columns=['fetal_health']).astype(np.float32)
y1=data1['fetal_health']

x2=data2.drop(columns=['fetal_health']).astype(np.float32)
y2=data2['fetal_health']

In [251]:
for column in list(x1):
  x1[column] = (x1[column] - x1[column].min()) / (x1[column].max() - x1[column].min())


for column in list(x2):
  x2[column] = (x2[column] - x2[column].min()) / (x2[column].max() - x2[column].min())

In [252]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.2,random_state=20, stratify=y1)
x_train2,x_test2,y_train2,y_test2=train_test_split(x2,y2,test_size=0.2,random_state=20, stratify=y2)

In [253]:
stream = DataStream(x_train2.to_numpy(), y_train2.to_numpy())
vfdt = HoeffdingTreeClassifier()

x_train1=x_train1.to_numpy()
y_train1=y_train1.to_numpy()
vfdt.fit(x_train1,y_train1)

# Setup variables to control loop and track performance

HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                        leaf_prediction='nba', max_byte_size=33554432,
                        memory_estimate_period=1000000, nb_threshold=0,
                        no_preprune=False, nominal_attributes=None,
                        remove_poor_atts=False, split_confidence=1e-07,
                        split_criterion='info_gain', stop_mem_management=False,
                        tie_threshold=0.05)

In [254]:
correct_cnt = 0
n_samples = 0

stream.restart()
stream = DataStream(x_test1.to_numpy(), y_test1.to_numpy())
predict_list=[]

while stream.has_more_samples():
  X, y = stream.next_sample()
  y_pred = vfdt.predict(X)
  predict_list.append(y_pred)
  if y[0] == y_pred[0]:
        correct_cnt += 1
  n_samples+=1

stream.restart()
stream= DataStream(x_test2.to_numpy(), y_test2.to_numpy())

while stream.has_more_samples():
  X, y = stream.next_sample()
  y_pred = vfdt.predict(X)
  predict_list.append(y_pred)
  if y[0] == y_pred[0]:
        correct_cnt += 1
  n_samples+=1


print('Hoeffding Tree accuracy: {}'.format(correct_cnt / n_samples))


Hoeffding Tree accuracy: 0.8486997635933806


In [255]:
from sklearn.metrics import classification_report
test_data = pd.concat([y_test1, y_test2],axis=0)
print(classification_report(test_data,predict_list))

              precision    recall  f1-score   support

           1       0.92      0.91      0.91       329
           2       0.61      0.76      0.68        59
           3       0.65      0.43      0.52        35

    accuracy                           0.85       423
   macro avg       0.73      0.70      0.70       423
weighted avg       0.85      0.85      0.85       423



### Further Training

In [256]:
while stream.has_more_samples():
  X, y = stream.next_sample()
  vfdt = vfdt.partial_fit(X, y)

In [257]:
correct_cnt = 0
n_samples = 0

stream.restart()
stream = DataStream(x_test1.to_numpy(), y_test1.to_numpy())
predict_list=[]

while stream.has_more_samples():
  X, y = stream.next_sample()
  y_pred = vfdt.predict(X)
  predict_list.append(y_pred)
  if y[0] == y_pred[0]:
        correct_cnt += 1
  n_samples+=1

stream.restart()
stream= DataStream(x_test2.to_numpy(), y_test2.to_numpy())

while stream.has_more_samples():
  X, y = stream.next_sample()
  y_pred = vfdt.predict(X)
  predict_list.append(y_pred)
  if y[0] == y_pred[0]:
        correct_cnt += 1
  n_samples+=1


print('Hoeffding Tree accuracy: {}'.format(correct_cnt / n_samples))

Hoeffding Tree accuracy: 0.8486997635933806


In [258]:
from sklearn.metrics import classification_report

In [259]:
test_data = pd.concat([y_test1, y_test2],axis=0)

In [261]:
print(classification_report(test_data,predict_list))

              precision    recall  f1-score   support

           1       0.92      0.91      0.91       329
           2       0.61      0.76      0.68        59
           3       0.65      0.43      0.52        35

    accuracy                           0.85       423
   macro avg       0.73      0.70      0.70       423
weighted avg       0.85      0.85      0.85       423

